In [23]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Import Packages

In [6]:
import duckdb
import plotly.express as px
import pandas as pd

# Connect to Local Database

In [44]:
# Create connection to local duckdb database
con = duckdb.connect('../db/duck.db')
%sql con --alias duckdb

# View Database

In [34]:
con.sql("SHOW DATABASES;")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ duck          │
└───────────────┘

In [35]:
con.sql("SHOW TABLES;")

┌───────────────────────────┐
│           name            │
│          varchar          │
├───────────────────────────┤
│ balance_sheet_assets      │
│ balance_sheet_equity      │
│ balance_sheet_liabilities │
│ cash_flow_statement       │
│ dividend                  │
│ earnings_calendar         │
│ eps_estimate              │
│ eps_history               │
│ features                  │
│ features_temp             │
│ income_statement          │
│ ohlcv                     │
│ pricing                   │
│ rank_score                │
│ sales_estimate            │
│ split                     │
│ split_adj                 │
│ symbol                    │
│ wolfe                     │
├───────────────────────────┤
│          19 rows          │
└───────────────────────────┘

# Create Features Table

In [ ]:
con.sql("""
    CREATE OR REPLACE TABLE features_raw (
        date DATE, 
        id VARCHAR,
        symbol VARCHAR,
        value DOUBLE, 
        PRIMARY KEY (date, id, symbol)
    );
""")

# Create Wolfe Ref Feature Table

In [12]:
con.sql("""
    --read csv data/wolfe/wolfe_feat_ref.csv and insert into wolfe table
    CREATE OR REPLACE TABLE wolfe AS (
        SELECT * FROM read_csv('../data/wolfe/wolfe_feat_ref.csv')
    );
""")

In [39]:
%%sql
select * from wolfe limit 5;

Running query in 'duckdb'

id,style_1,style_2,style_3,feature,name,description,dir,pub_yr,paper
1,Strategy,Profitability,Historic Profitability,BERRY,Berry ratio (gross profits/operating expenses),Calculated by subtracting total cost of goods sold from total sales and then dividing by total operating expense.,1,1999,"Berry Ratios: Their Use and Misuse, Journal of Global Transfer Pricing (Berry, 1999)"
2,Strategy,Profitability,Historic Profitability,CFRNOA,Cash flow return on net operating assets (CFRNOA),"Ratio of sum of net operating cash flow return and after-tax interest and related expenses to the sum of cash and short-term investments, total preferred stock, total common equity, total debt in current liabilities, and total long term debt.",1,2003,"Accrued Earnings and Growth: Implications for Future Profitability and Market Mispricing (Fairfield et al., 2003)"
3,Strategy,Profitability,Historic Profitability,CFROA,Cash flow return on asset (CFROA),Ratio of sum of net operating cash flow and after-tax interest and related expenses to the average of total assets and total assets lagged by 1 year.,1,2010,"A Better Three-Factor Model That Explains More Anomalies (Chen and Zhang, 2010)"
4,Strategy,Profitability,Historic Profitability,CFROE,Cash flow return on equity (CFROE),Ratio of rolling sum of operating net cash flow of last twelve months to the average of current total equity and total equity lagged by 1 year.,1,2010,"A Better Three-Factor Model That Explains More Anomalies (Chen and Zhang, 2010)"
5,Strategy,Profitability,Historic Profitability,GROSS_MARGIN,Gross profit margin,Calculated by dividing total gross profit by total sales.,1,2013,"The other side of value: The gross profitability premium (Novy-Marx, 2013)"


# Close Database Connection

In [46]:
con.close()